In [2]:
import os
import PIL
import random
import torchvision

from pathlib import Path
from torch.utils.data import Dataset, DataLoader
from torchvision.transforms import functional as f


def get_train_test_paths(test_ratio: float = 0.2):
    # extract the data from the dataset folder
    files = [file_name for file_name in
             Path(os.getcwd() + os.sep + 'Water Bodies Dataset' + os.sep + 'Images').rglob("*.jpg")]
    # randomize the order of the data
    random.shuffle(files)
    # separate test and train files
    first_train = int(test_ratio * len(files))
    test_path = files[:first_train]
    train_path = files[first_train:]
    return train_path, test_path


def get_mask_path(file_path):
    # gets source image path, returns mask path
    file_path = str(file_path).replace('Images', 'Masks')
    return file_path


class WaterDataset(Dataset):
    def __init__(self, path_list, transform_source=None, transform_both=None):
        self.sources = path_list
        self.transform_source = transform_source
        self.transform_both = transform_both

    def __len__(self):
        return len(self.sources)

    def __getitem__(self, index):
        img_path = self.sources[index]
        source = f.to_tensor(PIL.Image.open(img_path))
        label = f.to_tensor(PIL.Image.open(get_mask_path(img_path)).convert('L'))

        if self.transform_source:
            source = self.transform_source(source)
        if self.transform_both:
            source = self.transform_both(source)
            label = self.transform_both(label)
            label = (label < 0.5).float()

            assert len(label.unique()) <= 2, "threshold didn't work"

        return source, label


def get_train_test_loaders(batch_size, length):
    train_path, test_path = get_train_test_paths()
    train_loader = DataLoader(dataset=WaterDataset(train_path,
                                                   transform_both=torchvision.transforms.Resize((length, length))),
                              batch_size=batch_size,
                              shuffle=True)
    test_loader = DataLoader(dataset=WaterDataset(test_path,
                                                  transform_both=torchvision.transforms.Resize((length, length))),
                             batch_size=batch_size)

    return train_loader, test_loader

In [3]:
import pandas as pd

from Models import *
from torch import optim
from datetime import datetime
from tqdm.notebook import tqdm as tqdm
from WaterDataset import get_train_test_loaders
from sklearn.metrics import accuracy_score, f1_score


def fit_model(model, model_parameters, loss_function, optimizer, batch_size, image_normalized_length):
    # retrieve train and test files
    train_loader, test_loader = get_train_test_loaders(batch_size, image_normalized_length)
    device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
    # assign the model
    model = model(*model_parameters).to(device)
    # same importance for every pixel, same calculation for every pixel
    loss_reduction = 'sum'
    pos_weight = torch.ones([(image_normalized_length ** 2) * batch_size]).to(device)
    # set a loss function
    criterion = loss_function(reduction=loss_reduction, pos_weight=pos_weight)
    # set an optimizer
    optimizer = optimizer(model.parameters(), lr=0.001)
    # MODEL TRAINING
    model.train()
    for epoch in range(10):
        # start counting epoch duration
        epoch_start = datetime.now()
        # initiate epoch loss
        epoch_loss = 0
        # iterate through all data pairs
        for image, mask in tqdm(train_loader):
            # convert input pixel to tensor
            x = torch.tensor(image).float().to(device)
            # convert target to tensor
            tag = torch.tensor(mask, dtype=torch.float).flatten().to(device)
            # reset all gradients
            optimizer.zero_grad()
            # save current prediction
            prediction = model(x).view(-1)
            if len(prediction) != len(pos_weight):
                pos_weight = torch.ones([len(prediction)]).to(device)
                criterion = loss_function(reduction=loss_reduction, pos_weight=pos_weight)
            # activate loss function, calculate loss
            loss = criterion(prediction, tag)
            # back propagation
            loss.backward()
            optimizer.step()
            # update epoch loss
            epoch_loss += loss.item()
        # stop counting epoch duration
        epoch_end = datetime.now()
        epoch_seconds = (epoch_end - epoch_start).total_seconds()
        # MODEL EVALUATION
        model.eval()
        # collect predicted results and real results
        predicted, real = list(), list()
        for x, y in tqdm(test_loader):
            x = x.to(device)
            real.append(y)
            probabilities = model(x)
            batch_predicted = torch.argmax(probabilities, dim=1)
            predicted.append(batch_predicted)
        real = torch.cat(real).flatten()
        predicted = torch.cat(predicted).flatten().detach().cpu()
        # calculate accuracy and f1 score
        accuracy = accuracy_score(real, predicted)
        f1 = f1_score(real, predicted)
        # append results to csv file
        hyperparameters = {'Input Image Length': image_normalized_length,
                           'Hidden Layer Size': hidden_layer_size,
                           'Activation Function': model_parameters[2],
                           'Optimizer': optimizer,
                           'Loss Function': loss_function}
        df = pd.DataFrame({'Model Name': ['Hidden1'],
                           'Iteration': [epoch],
                           'Hyperparameters': str(hyperparameters),
                           'Loss': [epoch_loss],
                           'Accuracy': [accuracy],
                           'F1': [f1],
                           'Iteration Training Seconds': [epoch_seconds]})
        df.to_csv('Water_Bodies_Results.csv', index=False, mode='a', header=False)
        print(df)


if __name__ == '__main__':
    loss_func = nn.BCEWithLogitsLoss
    optimizers = (optim.Adam, optim.SGD)
    activation_funcs = (f.relu, f.leaky_relu, f.sigmoid)
    image_normalized_length = 100
    batch_size = 2
    hidden_layer_size = 50

    # train the model
    model_parameters = (image_normalized_length, hidden_layer_size, activation_funcs[0])
    fit_model(Hidden1, model_parameters, loss_func, optimizers[0], batch_size, image_normalized_length)


  0%|                                                                                                                                                                                                            | 0/1137 [00:00<?, ?it/s]C:\Users\nzams\AppData\Local\Temp/ipykernel_5828/1293153585.py:35: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  x = torch.tensor(image).float()
C:\Users\nzams\AppData\Local\Temp/ipykernel_5828/1293153585.py:37: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  tag = torch.tensor(mask, dtype=torch.float).flatten()
100%|█████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████

  Model Name  Iteration                                    Hyperparameters  \
0    Hidden1          0  "input_image_length": 5\n                     ...   

           Loss  Accuracy   F1  Iteration Training Seconds  
0  1.503022e+07  0.334067  0.0                   87.089792  


  0%|                                                                                                                                                                                                            | 0/1137 [00:00<?, ?it/s]C:\Users\nzams\AppData\Local\Temp/ipykernel_5828/1293153585.py:35: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  x = torch.tensor(image).float()
C:\Users\nzams\AppData\Local\Temp/ipykernel_5828/1293153585.py:37: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  tag = torch.tensor(mask, dtype=torch.float).flatten()
100%|█████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████

  Model Name  Iteration                                    Hyperparameters  \
0    Hidden1          1  "input_image_length": 5\n                     ...   

           Loss  Accuracy   F1  Iteration Training Seconds  
0  1.444846e+07  0.334067  0.0                  102.760746  


  0%|                                                                                                                                                                                                            | 0/1137 [00:00<?, ?it/s]C:\Users\nzams\AppData\Local\Temp/ipykernel_5828/1293153585.py:35: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  x = torch.tensor(image).float()
C:\Users\nzams\AppData\Local\Temp/ipykernel_5828/1293153585.py:37: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  tag = torch.tensor(mask, dtype=torch.float).flatten()
100%|█████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████

...

  Model Name  Iteration                                    Hyperparameters  \
0    Hidden1          7  "input_image_length": 5\n                     ...   

           Loss  Accuracy   F1  Iteration Training Seconds  
0  1.421297e+07  0.334067  0.0                   84.454644  


  0%|                                                                                                                                                                                                            | 0/1137 [00:00<?, ?it/s]C:\Users\nzams\AppData\Local\Temp/ipykernel_5828/1293153585.py:35: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  x = torch.tensor(image).float()
C:\Users\nzams\AppData\Local\Temp/ipykernel_5828/1293153585.py:37: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  tag = torch.tensor(mask, dtype=torch.float).flatten()
100%|█████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████

  Model Name  Iteration                                    Hyperparameters  \
0    Hidden1          8  "input_image_length": 5\n                     ...   

           Loss  Accuracy   F1  Iteration Training Seconds  
0  1.421296e+07  0.334067  0.0                   82.809281  


  0%|                                                                                                                                                                                                            | 0/1137 [00:00<?, ?it/s]C:\Users\nzams\AppData\Local\Temp/ipykernel_5828/1293153585.py:35: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  x = torch.tensor(image).float()
C:\Users\nzams\AppData\Local\Temp/ipykernel_5828/1293153585.py:37: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  tag = torch.tensor(mask, dtype=torch.float).flatten()
100%|█████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████

  Model Name  Iteration                                    Hyperparameters  \
0    Hidden1          9  "input_image_length": 5\n                     ...   

           Loss  Accuracy   F1  Iteration Training Seconds  
0  1.421294e+07  0.334067  0.0                   80.899928  
